# 📊 Análise Exploratória de Dados de da Rede Sonda 🌦️

## 📌 Introdução
Este notebook realiza uma análise exploratória dos dados meteorológicos coletados de diversas estações. O objetivo é entender a estrutura dos dados, avaliar sua qualidade e identificar padrões relevantes.

## 📂 Fonte dos Dados
- Arquivos CSV formatados armazenados no ftp
- Contêm medições de variáveis meteorológicas, solarimétricas e cameras.

## 🔍 Objetivos da Análise
1. **Carregar e explorar os dados**: verificar onde os dados estão armazenados, seu formato e estrutura.
2. **Dimensionamento e variáveis disponíveis**: entender o tamanho dos arquivos, número de registros e colunas.
3. **Análise temporal dos dados disponíveis**: identificar o período coberto e eventuais lacunas temporais.
4. **Visualização da distribuição espacial das estações**: verificar a abrangência geográfica das medições.
5. **Exploração inicial de distribuições**: histogramas e estatísticas básicas das variáveis.
6. **Análise de qualidade dos dados** *(última etapa)*: identificar valores ausentes, inconsistências e flags de qualidade.

### 1. Carregar e Explorar os Dados
Vamos começar listando o tamanho da base de dados que estão no diretório do ftp.

In [1]:
# Diretório onde os arquivos estão localizados
DIRETORIO = '../sonda/dados_formatados/'

In [2]:
# Exibe o tamanho de cada arquivo no diretório ordenado por tamanho de forma decrescente
!du -h --max-depth=1 {DIRETORIO} | sort -rh

11G	../sonda/dados_formatados/
1,2G	../sonda/dados_formatados/BRB
1011M	../sonda/dados_formatados/PTR
989M	../sonda/dados_formatados/FLN
959M	../sonda/dados_formatados/PMA
759M	../sonda/dados_formatados/JOI
722M	../sonda/dados_formatados/CPA
714M	../sonda/dados_formatados/SMS
686M	../sonda/dados_formatados/SLZ
613M	../sonda/dados_formatados/NAT
536M	../sonda/dados_formatados/CGR
464M	../sonda/dados_formatados/SBR
413M	../sonda/dados_formatados/TMA
365M	../sonda/dados_formatados/MCL
349M	../sonda/dados_formatados/ORN
300M	../sonda/dados_formatados/UBE
284M	../sonda/dados_formatados/BJL
175M	../sonda/dados_formatados/TLG
174M	../sonda/dados_formatados/CAI
171M	../sonda/dados_formatados/CTB
55M	../sonda/dados_formatados/CBA
196K	../sonda/dados_formatados/TRI
196K	../sonda/dados_formatados/SPK
196K	../sonda/dados_formatados/SCR
196K	../sonda/dados_formatados/RLM
196K	../sonda/dados_formatados/OPO
196K	../sonda/dados_formatados/MDS
196K	../sonda/dados_formatados/LEB
196K	../sonda/dados_form

Existem 3 tipos de dados:
- Dados Meteorológicos
- Dados Solarimétricos
- Dados Anemometricos

In [3]:
import glob

# listar todos os dados Meteorológicos usando o glob só para o tipo de arquivo .csv
dados_metereologicos = glob.glob(DIRETORIO + "*/Meteorologicos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_metereologicos = [arquivo for arquivo in dados_metereologicos if 'YYYY_MM' not in arquivo]

# listar todos os dados de Solarimétricos usando o glob só para o tipo de arquivo .csv
dados_solarimetricos = glob.glob(DIRETORIO + "*/Solarimetricos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_solarimetricos = [arquivo for arquivo in dados_solarimetricos if 'YYYY_MM' not in arquivo]

# listar todos os dados de Anemometricos usando o glob só para o tipo de arquivo .csv
dados_anemometricos = glob.glob(DIRETORIO + "*/Anemometricos/**/*.csv", recursive=True)
# Remove arquivos que contenham 'YYYY_MM_MD_DQC'
dados_anemometricos = [arquivo for arquivo in dados_anemometricos if 'YYYY_MM' not in arquivo]

In [4]:
# Listar a quantidade de arquivos em cada categoria
print(f"Quantidade de arquivos Meteorologicos: {len(dados_metereologicos)}")
print(f"Quantidade de arquivos Solarimetricos: {len(dados_solarimetricos)}")
print(f"Quantidade de arquivos Anemometricos: {len(dados_anemometricos)}")

Quantidade de arquivos Meteorologicos: 1036
Quantidade de arquivos Solarimetricos: 1022
Quantidade de arquivos Anemometricos: 0


In [5]:
import polars as pl

def verifica_schema(lista_arquivos):

    schema_dif = {}
    # Lê os schema do primeiro arquivo e compara com os demais
    schema = pl.read_csv(lista_arquivos[0]).schema
    for arquivo in lista_arquivos[1:]:
        schema_arquivo = pl.read_csv(arquivo).schema
        if schema_arquivo != schema:
            schema_dif[arquivo] = schema_arquivo

    # Caso exista diferença de schema, exibe os arquivos e os schemas diferentes
    if schema_dif:
        print("Arquivos com schema diferente:")
        for arquivo, schema_arv in schema_dif.items():
            print(f"Arquivo: {arquivo}")
            print(f"Schema do arquivo: {schema_arv}")
        print(f"Schema   Original: {schema}")
        return schema, schema_dif
    else:
        print("Todos os arquivos possuem o mesmo schema")
        print(f"Schema: {schema}")
    return schema, schema_dif

In [6]:
print('Arquivos Meteorologicos')
schema_meteorologico, meteorologico_remover = verifica_schema(dados_metereologicos)

print('\nArquivos Solarimetricos')
schema_solarimetrico, solar_remover = verifica_schema(dados_solarimetricos)

Arquivos Meteorologicos
Todos os arquivos possuem o mesmo schema
Schema: Schema({'acronym': String, 'timestamp': String, 'year': Int64, 'day': Int64, 'min': Int64, 'tp_sfc': String, 'humid_sfc': String, 'press': String, 'rain': String, 'ws10_avg': String, 'ws10_std': String, 'wd10_avg': String, 'wd10_std': String})

Arquivos Solarimetricos
Arquivos com schema diferente:
Arquivo: ../sonda/dados_formatados/PMA/Solarimetricos/2016/PMA_2016_06_SD_formatado.csv
Schema do arquivo: Schema({'acronym': String, 'timestamp': String, 'year': Float64, 'day': Float64, 'min': Float64, 'glo_avg': String, 'glo_std': String, 'glo_max': String, 'glo_min': String, 'dif_avg': String, 'dif_std': String, 'dif_max': String, 'dif_min': String, 'par_avg': String, 'par_std': String, 'par_max': String, 'par_min': String, 'lux_avg': String, 'lux_std': String, 'lux_max': String, 'lux_min': String, 'dir_avg': String, 'dir_std': String, 'dir_max': String, 'dir_min': String, 'lw_avg': String, 'lw_std': String, 'lw_max

In [7]:
# Remover arquivos que possuem schema diferente
dados_metereologicos = [arquivo for arquivo in dados_metereologicos if arquivo not in meteorologico_remover.keys()]
dados_solarimetricos = [arquivo for arquivo in dados_solarimetricos if arquivo not in solar_remover.keys()]

In [8]:
# Função para leitura rápida de arquivos
def ler_csv_rapido(file, schema):
    return pl.scan_csv(
        file, 
        skip_rows_after_header=1,  # Pula a linha logo após o cabeçalho
        schema_overrides=schema,  # Utiliza o schema do primeiro arquivo
    )

In [ ]:
import os

# Verifica se arquivo já existe em ../sonda/ caso sim, leia o arquivo, caso não, leia o arqui
if os.path.exists('../sonda/meteorologicos.parquet'):
    df_meteorologico = pl.read_parquet('../sonda/meteorologicos.parquet')
else:
    # Lê todos os arquivos de dados meteorológicos
    df_meteorologico = pl.concat([ler_csv_rapido(file, schema_meteorologico) for file in dados_metereologicos])
    # Materializar os dados na memória
    df_meteorologico = df_meteorologico.collect()
    # Salva os dados em um arquivo parquet
    df_meteorologico.write_parquet('../sonda/meteorologicos.parquet', use_pyarrow=True)

# Verifica se arquivo já existe em ../sonda/ caso sim, leia o arquivo, caso não, leia o arqui
if os.path.exists('../sonda/solarimetricos.parquet'):
    df_solarimetrico = pl.read_parquet('../sonda/solarimetricos.parquet')
else:
    # Configuração global do número de threads
    os.environ["POLARS_MAX_THREADS"] = "5" 
    # Lê todos os arquivos de dados solarimétricos
    df_solarimetrico = pl.concat([ler_csv_rapido(file, schema_solarimetrico) for file in dados_solarimetricos])
    # Materializar os dados na memória de maneira apropriada usando poucas threads
    df_solarimetrico = df_solarimetrico.collect()
    # Salva os dados em um arquivo parquet
    df_solarimetrico.write_parquet('../sonda/solarimetricos.parquet', use_pyarrow=True)